In [1]:
from maap.maap import MAAP
#maap = MAAP(maap_host='api.ops.maap-project.org')
maap = MAAP()

In [105]:
maap._MAAP_HOST

'api.maap-project.org'

In [2]:
#!pip install -U -r /projects/Developer/icesat2_boreal/dps/requirements_main.txt

# Launch DPS for build_stack.py

In [3]:
# For some reason this is needed to get s3fs to work in ExtractUtils
# this upgrades to 0.3.4 even though we already specify this version in requirements_main...
#!pip install s3fs --upgrade

In [27]:
import os
import geopandas
import pandas as pd
import glob
import datetime
!pip install xmltodict
import xmltodict
import sys
sys.path.append('/projects/code/icesat2_boreal/lib')
import ExtractUtils

In [5]:
import s3fs
s3fs.__version__

'2023.12.0'

# Test (locally) the script for DPS

##### To run build_stack.py across a tiled raster dataset you need a bunch of args that we'll gather into a dictionary

s3 you need to have a vector footprint of that dataset

In [22]:
TILE_NUM = 1274

#### Dictionary preparation makes this script very flexible and transferable to another s3 dataset
This dictionary is specific to the ESA Worldcover dataset.  
To run '`build_stack.py` across another dataset, just prepare another dictionary here and everything below should be exactly the same.  

In [23]:
# ESA Worldcover 2020
BUILD_STACK_DICT = {
            #'INDEX_FN': '/projects/my-public-bucket/boreal_tiles_v003.gpkg',
            'INDEX_FN': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
            'ID_COL_NAME': 'tile_num',
            'TILE_NUM':TILE_NUM,
            'INDEX_LYR': 'boreal_tiles_v003',
            # Worldcover data is accessed via its footprint, with a 's3_path' col identifying the s3 locations of each tile
            'RASTER_NAME': 'esa_worldcover_v100_2020',
            #'COVAR_TILE_FN': '/projects/my-public-bucket/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg',
            'COVAR_TILE_FN': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg',
            'IN_COVAR_S3_COL': 's3_path',
            'OUTDIR': '/projects/my-public-bucket/DPS_ESA_LC',
            'NODATA_VAL': 0,
            'OUTPUT_CLIP_COG_FN':'',
            'CREDENTIALS_FN': None
        }

In [24]:
# INDEX clip shapes should be in equal area
# COVAR_TILE_FN tiles (of raster COGs) should be in 4326
args = f"\
    --in_tile_fn {BUILD_STACK_DICT['INDEX_FN']} \
    --in_tile_id_col {BUILD_STACK_DICT['ID_COL_NAME']} \
    --in_tile_num {BUILD_STACK_DICT['TILE_NUM']} \
    --tile_buffer_m 0 \
    --in_tile_layer {BUILD_STACK_DICT['INDEX_LYR']} \
    -o {BUILD_STACK_DICT['OUTDIR']} \
    --topo_off \
    --covar_src_name {BUILD_STACK_DICT['RASTER_NAME']} \
    --covar_tile_fn {BUILD_STACK_DICT['COVAR_TILE_FN']} \
    --in_covar_s3_col {BUILD_STACK_DICT['IN_COVAR_S3_COL']} \
    --input_nodata_value {BUILD_STACK_DICT['NODATA_VAL']} \
    --shape 3000 \
    --clip"

In [25]:
!echo python /projects/code/icesat2_boreal/lib/build_stack.py $args

python /projects/code/icesat2_boreal/lib/build_stack.py --in_tile_fn https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg --in_tile_id_col tile_num --in_tile_num 1274 --tile_buffer_m 0 --in_tile_layer boreal_tiles_v003 -o /projects/my-public-bucket/DPS_ESA_LC --topo_off --covar_src_name esa_worldcover_v100_2020 --covar_tile_fn https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg --in_covar_s3_col s3_path --input_nodata_value 0 --shape 3000 --clip


In [8]:
DPS_INPUT_TILE_NUM_LIST = []

### Use MAAP Registration call in notebook chunk to register DPS algorithm
 - We need to register a DPS algorithm called `run_build_stack_LC` before proceeding to the chunks below...

In [139]:
maap.register_algorithm_from_yaml_file("/projects/code/icesat2_boreal/dps/registered/run_build_stack.yml").text

'{"code": 200, "message": {"id": "0a25ec97c13cba1eb525b665d6cf32e48ce8c5fa", "short_id": "0a25ec97", "created_at": "2023-12-07T02:01:00.000+00:00", "parent_ids": ["48378de9a13c91ef1fe922456e6711ddd837ee01"], "title": "Registering algorithm: run_build_stack", "message": "Registering algorithm: run_build_stack", "author_name": "root", "author_email": "root@e49d5cea4b76", "authored_date": "2023-12-07T02:01:00.000+00:00", "committer_name": "root", "committer_email": "root@e49d5cea4b76", "committed_date": "2023-12-07T02:01:00.000+00:00", "trailers": {}, "web_url": "https://repo.maap-project.org/root/register-job-hysds-v4/-/commit/0a25ec97c13cba1eb525b665d6cf32e48ce8c5fa", "stats": {"additions": 1, "deletions": 1, "total": 2}, "status": "pending", "project_id": 3, "last_pipeline": {"id": 10345, "iid": 616, "project_id": 3, "sha": "0a25ec97c13cba1eb525b665d6cf32e48ce8c5fa", "ref": "main", "status": "pending", "source": "push", "created_at": "2023-12-07T02:01:02.128Z", "updated_at": "2023-12-0

In [144]:
#!aws s3 rm --recursive s3://maap-ops-workspace/montesano/dps_output/run_build_stack/LC_ESA_WC_2020
!aws s3 mv s3://maap-ops-workspace/montesano/dps_output/run_build_stack/LC_ESA_WC_2020 s3://maap-ops-workspace/montesano/dps_output/run_build_stack/build_stack_v2023

fatal error: An error occurred (404) when calling the HeadObject operation: Key "montesano/dps_output/run_build_stack/LC_ESA_WC_2020" does not exist


# Build a DPS list

In [36]:
#HLS_tindex_master_fn = "s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS/fall2022/HLS_stack_2022_v2/HLS_tindex_master.csv".replace('s3://maap-ops-workspace/shared/nathanmthomas', '/projects/my-public-bucket')
Topo_tindex_master_fn = "s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv" #.replace('s3://maap-ops-workspace/shared/nathanmthomas', '/projects/my-public-bucket')

# Spring 2022 c2020 composite - use this as template set of tiles (5241 tiles)
HLS_EXISTING_TINDEX_FN = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS/c2020/HLS_stack_2023_v1/HLS_tindex_master.csv'

s3 = s3fs.S3FileSystem(anon=True)

DPS_INPUT_TILE_NUM_LIST = []
for tindex_fn in [HLS_EXISTING_TINDEX_FN, Topo_tindex_master_fn]:
    tindex = pd.read_csv(tindex_fn, storage_options={'anon':True})
    
    print(len(tindex['tile_num'].values))
    
    if len(tindex['tile_num'].values) > len(DPS_INPUT_TILE_NUM_LIST):
        print(f"{len(tindex['tile_num'].values)} tiles in {tindex_fn}")
        print(f'{os.path.basename(tindex_fn)} has the most tiles. Using this for DPS tiles list.')
        DPS_INPUT_TILE_NUM_LIST = tindex['tile_num'].values

len(DPS_INPUT_TILE_NUM_LIST)

5241
5241 tiles in s3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS/c2020/HLS_stack_2023_v1/HLS_tindex_master.csv
HLS_tindex_master.csv has the most tiles. Using this for DPS tiles list.
5220


5241

In [43]:
RUN_IRREGULAR_TILES = False

IRREGULAR_TILES_FN = '/projects/my-public-bucket/DPS_tile_lists/HLS_irregular_tiles.csv'

if RUN_IRREGULAR_TILES:
    DPS_INPUT_TILE_NUM_LIST = []
    print("Running DPS on irregular tiles that forces a shape of 3000...")
    df = pd.read_csv(IRREGULAR_TILES_FN)
    tile_num_list = []
    for i in df["IrregularTiles"]:
        tile_num = int(i.split('_')[9])
        DPS_INPUT_TILE_NUM_LIST.append(tile_num)
    print(len(DPS_INPUT_TILE_NUM_LIST))
    print(DPS_INPUT_TILE_NUM_LIST)

In [42]:
len(DPS_INPUT_TILE_NUM_LIST)

5241

#### Note: make sure the `in_params_dict` coincides with the args of `build_stack.py`

In [29]:
#'INDEX_FN': '/projects/my-public-bucket/boreal_tiles_v003.gpkg',
in_params_dict = {
            'covar_tile_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg',
            'in_tile_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
            'covar_tile_fn': 'footprints_v100_2020_v100_2020_map-s3.gpkg',
            'in_tile_fn': 'boreal_tiles_v003.gpkg',
            'in_tile_id_col': 'tile_num',
            'in_tile_num':"",
            'tile_buffer_m': 0,
            'in_tile_layer': 'boreal_tiles_v003',
            #'output_dir': 'dummy',  # a dummy dir so i dont have to change the .yaml now
            #'topo_off': 'dummy', # functionality to turn off build a 'topo' stack is FALSE by default - so we have to set this flag (which turns OFF topo building) - this is hardcoded in the .sh now - if we want to build a topo stack with this script, need to learn how to set this flag with params dict 
            'covar_src_name': 'esa_worldcover_v100_2020',
            'in_covar_s3_col': 's3_path',
            'input_nodata_value': 0,
            'shape': 3000
            #'clip': 'dummy' # this is hardcoded in the .sh now - if we want to build a topo stack with this script, need to learn how to set this flag with params dict 
        }

In [30]:
in_params_dict

{'covar_tile_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/analyze_agb/footprints_v100_2020_v100_2020_map-s3.gpkg',
 'in_tile_url': 'https://maap-ops-workspace.s3.amazonaws.com/shared/nathanmthomas/boreal_tiles_v003.gpkg',
 'covar_tile_fn': 'footprints_v100_2020_v100_2020_map-s3.gpkg',
 'in_tile_fn': 'boreal_tiles_v003.gpkg',
 'in_tile_id_col': 'tile_num',
 'in_tile_num': '',
 'tile_buffer_m': 0,
 'in_tile_layer': 'boreal_tiles_v003',
 'covar_src_name': 'esa_worldcover_v100_2020',
 'in_covar_s3_col': 's3_path',
 'input_nodata_value': 0,
 'shape': 3000}

## Run a DPS job across the list

In [148]:
# MAAP algorithm version name
IDENTIFIER='LC_ESA_WC_2020'
MAAP_VERSION = 'build_stack_v2023'
ALGO_ID = "run_build_stack"
USER = 'montesano'
WORKER_TYPE = 'maap-dps-worker-8gb'

In [149]:
RUN_NAME = IDENTIFIER

In [150]:
%%time

import json

submit_results_df_list = []
len_input_list = len(DPS_INPUT_TILE_NUM_LIST)
print(f"# of input tiles for DPS: {len_input_list}")

for i, INPUT_TILE_NUM in enumerate(DPS_INPUT_TILE_NUM_LIST):
    
    DPS_num = i+1
    
    # Update the in_params_dict with th current INPUT_TILE_NUM
    in_params_dict['in_tile_num'] = INPUT_TILE_NUM
    
    submit_result = maap.submitJob(
            identifier=IDENTIFIER,
            algo_id=ALGO_ID,
            version=MAAP_VERSION,
            username=USER, # username needs to be the same as whoever created the workspace
            queue=WORKER_TYPE,
            **in_params_dict
        )
    
    #print(submit_result)
    #break
    
    # Build a dataframe of submission details
    submit_result_df = pd.DataFrame( 
        {
                'dps_num':[DPS_num],
                'tile_num':[INPUT_TILE_NUM],
                'submit_time':[datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%s')],
                'dbs_job_hour': [datetime.datetime.now().hour],
                'algo_id': [ALGO_ID],
                'user': [USER],
                'worker_type': [WORKER_TYPE],
                'job_id': [submit_result.id],
                'submit_status': [submit_result.status],
            
        } 
    )
    
    # Append to a list of data frames of submission results
    submit_results_df_list.append(submit_result_df)
    
    if DPS_num in [1, 5, 10, 50, 100, 250, 500, 750, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 7000, 9000, 11000, 13000, 15000, 17000, 19000, 21000, 24000, len_input_list]:
        print(f"DPS run #: {DPS_num}\t| tile num: {INPUT_TILE_NUM}\t| submit status: {submit_result.status}\t| job id: {submit_result.id}") 
        
# Build a final submission results df and save
submit_results_df = pd.concat(submit_results_df_list)
submit_results_df['run_name'] = RUN_NAME
nowtime = pd.Timestamp.now().strftime('%Y%m%d%H%M')
print(f"Current time:\t{nowtime}")
submit_results_df.to_csv(f'/projects/my-public-bucket/dps_submission_results/DPS_{ALGO_ID}_{RUN_NAME}_submission_results_{len_input_list}_{nowtime}.csv')
submit_results_df.info()


# of input tiles for DPS: 5241
DPS run #: 1	| tile num: 4085	| submit status: success	| job id: 8d815eeb-4149-4ff9-b34f-99e1406b4d50
DPS run #: 5	| tile num: 1096	| submit status: success	| job id: f1403aa7-1ff2-42fc-af21-dbf2132a8879
DPS run #: 10	| tile num: 572	| submit status: success	| job id: ab711ff7-5cc5-4143-b754-14cd2b911298
DPS run #: 50	| tile num: 3096	| submit status: success	| job id: 939f3537-9a0c-4f72-8da0-14a32d1edefa
DPS run #: 100	| tile num: 2795	| submit status: success	| job id: d012f0eb-96e5-4756-9b93-fc1592c987df
DPS run #: 250	| tile num: 196	| submit status: success	| job id: 06b5177c-1c44-4d6d-a406-6ff23f88b4c1
DPS run #: 500	| tile num: 3618	| submit status: success	| job id: e84e422b-fe8a-4ac7-92fc-f01c38fc8861
DPS run #: 750	| tile num: 2007	| submit status: success	| job id: 38141138-a319-4345-ba80-6736190b711f
DPS run #: 1000	| tile num: 2742	| submit status: success	| job id: 110d176f-2e0b-4d72-946f-7817b2cba169
DPS run #: 1500	| tile num: 4017	| submi

After almost any DPS job, you have to assess what succeeded and failed. This involves:
1. building a table of job status based on job ids captured in the job_results_df from the DPS run chunk (this takes 40 mins for ~47k jobs) --> this tells you how many jobs failed
2. merging the job status table with the job results df --> this tells you which specific granules (or tile nums) failed
3. building another input list of granules for a follow-up DPS
## Assess DPS results
Build a table of job status based on job id - how many jobs failed?

In [132]:
import importlib
import ExtractUtils
importlib.reload(ExtractUtils)

NASA MAAP


<module 'ExtractUtils' from '/projects/code/icesat2_boreal/lib/ExtractUtils.py'>

In [121]:
from maap.dps.dps_job import DPSJob
job = DPSJob()
job.id = submit_result.id #'d21b4131-f199-4a3f-922f-a51ed6a7ceca'
job.retrieve_status

<bound method DPSJob.retrieve_status of {'job_id': 'd21b4131-f199-4a3f-922f-a51ed6a7ceca', 'status': None, 'machine_type': None, 'architecture': None, 'machine_memory_size': None, 'directory_size': None, 'operating_system': None, 'job_start_time': None, 'job_end_time': None, 'job_duration_seconds': None, 'cpu_usage': None, 'cache_usage': None, 'mem_usage': None, 'max_mem_usage': None, 'swap_usage': None, 'read_io_stats': None, 'write_io_stats': None, 'sync_io_stats': None, 'async_io_stats': None, 'total_io_stats': None, 'error_details': None, 'response_code': None, 'outputs': []}>

In [135]:
print(submit_result)

{'job_id': 'cdc12d74-48ae-48d7-a8b8-541d90e35e0b', 'status': 'success', 'machine_type': None, 'architecture': None, 'machine_memory_size': None, 'directory_size': None, 'operating_system': None, 'job_start_time': None, 'job_end_time': None, 'job_duration_seconds': None, 'cpu_usage': None, 'cache_usage': None, 'mem_usage': None, 'max_mem_usage': None, 'swap_usage': None, 'read_io_stats': None, 'write_io_stats': None, 'sync_io_stats': None, 'async_io_stats': None, 'total_io_stats': None, 'error_details': None, 'response_code': 200, 'outputs': []}


In [145]:
#maap.getJobStatus(submit_result.id)

In [133]:
%%time
running_list = []
fails_list = []
success_list = []
for DPS_DATETIME in [nowtime]:
    for fn in LIST_SUBMISSIONS:
        if DPS_DATETIME in fn and not 'job_status' in fn:
            
            DPS_alg_id = os.path.basename(fn.split('_submission_results_')[0].replace('DPS_',''))
            thentime = fn.split('_')[-1].replace('.csv','')
            print(f'DPS alg:\t\t{DPS_alg_id}')
            print(f'DPS run name:\t\t{RUN_NAME}')
            print(f'DPS launch time:\t{thentime}')
            
            # Build job status table
            df_jstatus = ExtractUtils.BUILD_TABLE_JOBSTATUS(pd.read_csv(fn))
            
            # Save job status table
            df_jstatus.to_csv(f'/projects/my-public-bucket/dps_submission_results/DPS_{RUN_NAME}_submission_results_job_status_{len(df_jstatus)}_{thentime}.csv')

            # Get current fails df and append to list
            df_jstatus['run_type'] = RUN_NAME

            running_list.append(df_jstatus[ (df_jstatus['status'] == 'Running') ] )
            fails_list.append(  df_jstatus[ (df_jstatus['status'] == 'Failed') ] )
            success_list.append(df_jstatus[ (df_jstatus['status'] == 'Succeeded') ] )
            
df_all_running = pd.concat(running_list)          
df_all_fails =   pd.concat(fails_list)
df_all_success = pd.concat(success_list)

ValueError: No objects to concatenate

In [85]:
%%time
LIST_SUBMISSIONS = sorted(glob.glob(f'/projects/my-public-bucket/dps_submission_results/DPS_{ALGO_ID}_{RUN_NAME}_submission_results_*.csv'),key=ExtractUtils.func, reverse=True)
for DPS_DATETIME in [nowtime]:
    for fn in LIST_SUBMISSIONS:
        if DPS_DATETIME in fn and not 'job_status' in fn:
            DPS_alg_id = os.path.basename(fn.split('_submission_results_')[0].replace('DPS_',''))
            thentime = fn.split('_')[-1].replace('.csv','')
            print(f'DPS alg:\t\t{DPS_alg_id}')
            print(f'DPS launch time:\t{thentime}')
            z = ExtractUtils.BUILD_TABLE_JOBSTATUS(pd.read_csv(fn))
            # Save job status table
            z.to_csv(f'/projects/my-public-bucket/dps_submission_results/DPS_{IDENTIFIER}_submission_results_job_status_{len(z)}_{thentime}.csv')

DPS alg:		run_build_stack_LC_build_stack_LC
DPS launch time:	202312061353


InvalidURL: Invalid URL 'https:///api/dps/job/9c7f52e7-3033-4c0c-bd30-8372b330a11d/status': No host supplied

In [179]:
z[z['wps:Status'] =='Succeeded'].tile_num.to_list()

[24389]

In [46]:
xmltodict.parse(maap.getJobResult(z[z['wps:Status'] =='Failed'].iloc[43].job_id).content)

{'wps:Result': {'@xmlns:ows': 'http://www.opengis.net/ows/2.0',
  '@xmlns:schemaLocation': 'http://schemas.opengis.net/wps/2.0/wps.xsd',
  '@xmlns:wps': 'http://www.opengis.net/wps/2.0',
  '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
  'wps:JobID': '20d88b45-eb25-48e6-b6f0-477ca59f3bb6',
  'wps:Output': {'@id': 'traceback',
   'wps:Data': "+ export HOME=/root\n+ HOME=/root\n+ exec /docker-stats-on-exit-shim _docker_stats.json /app/dps_wrapper.sh /app/icesat2_boreal/dps/alg_3-1-3/run.sh footprints_v100_2020_v100_2020_map-s3.gpkg boreal_tiles_v003.gpkg tile_num 2223 0 boreal_tiles_v003 esa_worldcover_v100_2020 s3_path 0 3000\n+ /app/icesat2_boreal/dps/alg_3-1-3/run.sh footprints_v100_2020_v100_2020_map-s3.gpkg boreal_tiles_v003.gpkg tile_num 2223 0 boreal_tiles_v003 esa_worldcover_v100_2020 s3_path 0 3000\n+ unset PROJ_LIB\n+ mkdir output\n+++ dirname /app/icesat2_boreal/dps/alg_3-1-3/run.sh\n++ cd /app/icesat2_boreal/dps/alg_3-1-3\n++ pwd -P\n+ basedir=/app/icesat2_boreal/

## Update the DPS input tiles list with only the tiles that failed - then run the DPS submit chunk above

In [160]:
DPS_INPUT_TILE_NUM_LIST = z[z['wps:Status'] =='Failed'].tile_num.to_list()
len(DPS_INPUT_TILE_NUM_LIST)
DPS_INPUT_TILE_NUM_LIST

[2917, 980, 25175, 25740, 2981, 3879]

In [162]:
STRAGGLER_TILE_LIST = [3269, 3088, 2917, 2513, 2228, 1554, 1274, 1200, 620, 980, 790, 23828, 24389, 24108, 23501, 23830, 108, 25175, 25458, 25740, 163, 28967, 1172, 1431, 2981, 3531, 4463, 3879, 4207, 4445, 2888, 3790]
DPS_INPUT_TILE_NUM_LIST = STRAGGLER_TILE_LIST
len(DPS_INPUT_TILE_NUM_LIST)

32

In [169]:
FINISHED_LIST = z[z['wps:Status'] =='Succeeded'].tile_num.to_list()
DPS_INPUT_TILE_NUM_LIST = set(STRAGGLER_TILE_LIST) - set(FINISHED_LIST)
len(DPS_INPUT_TILE_NUM_LIST)

6

In [174]:
STRAGGLER_TILE_LIST2 = [3855,24389]
DPS_INPUT_TILE_NUM_LIST = STRAGGLER_TILE_LIST2

In [155]:
USER = 'montesano'

DPS_IDENTIFIER = f'{MAAP_VERSION}/{IDENTIFIER}'
DPS_MONTH_LIST = '01 02 03 04 05 06 07 08 09 10 11 12'        
TYPE = 'LC'

DPS_DAY_MIN = 1
OUTDIR = f'/projects/my-public-bucket/DPS_tile_lists/{ALGO_ID}/{DPS_IDENTIFIER}'
!mkdir -p $OUTDIR

# Build tindex
args = f"--RETURN_DUPS \
--user {USER} \
--dps_identifier {DPS_IDENTIFIER} \
-alg_name {ALGO_ID} \
-t {TYPE} \
-y 2023 \
--dps_month_list {DPS_MONTH_LIST} \
-d_min {DPS_DAY_MIN} \
--outdir {OUTDIR}"

!time echo python /projects/code/icesat2_boreal/lib/build_tindex_master.py $args

python /projects/code/icesat2_boreal/lib/build_tindex_master.py --RETURN_DUPS --user montesano --dps_identifier build_stack_v2023/LC_ESA_WC_2020 -alg_name run_build_stack -t LC -y 2023 --dps_month_list 01 02 03 04 05 06 07 08 09 10 11 12 -d_min 1 --outdir /projects/my-public-bucket/DPS_tile_lists/build_stack_v2023/LC_ESA_WC_2020

real	0m0.000s
user	0m0.000s
sys	0m0.000s


In [159]:
df = pd.read_csv('/projects/my-public-bucket/DPS_tile_lists/build_stack_v2023/LC_ESA_WC_2020/LC_tindex_master_duplicates.csv')

In [160]:

df['tile_num'] = df['file'].str.split('_', expand=True)[4].str.strip('_cog.tif')
df.head()

,Unnamed: 0,s3_path,local_path,file,tile_num,creation time,status
0,2826,s3://maap-ops-workspace/montesano/dps_output/r...,/projects/my-private-bucket/dps_output/run_bui...,esa_worldcover_v100_2020_874_cog.tif,874,2023-12-07 06:39:10,dropped
1,2825,s3://maap-ops-workspace/montesano/dps_output/r...,/projects/my-private-bucket/dps_output/run_bui...,esa_worldcover_v100_2020_939_cog.tif,939,2023-12-07 06:39:09,dropped
2,2824,s3://maap-ops-workspace/montesano/dps_output/r...,/projects/my-private-bucket/dps_output/run_bui...,esa_worldcover_v100_2020_30505_cog.tif,30505,2023-12-07 06:39:09,dropped
3,2823,s3://maap-ops-workspace/montesano/dps_output/r...,/projects/my-private-bucket/dps_output/run_bui...,esa_worldcover_v100_2020_1191_cog.tif,1191,2023-12-07 06:39:09,dropped
4,2822,s3://maap-ops-workspace/montesano/dps_output/r...,/projects/my-private-bucket/dps_output/run_bui...,esa_worldcover_v100_2020_26826_cog.tif,26826,2023-12-07 06:39:09,dropped
